# How to build a database
An important element of the COSIMA cookbook is that model output metadata is held in a database. This database allows for easy querying of the data so that we so that variables can be loaded with a single command.

The current version of the cookbook (as of May 2019) has been upgraded to allow for multiple databases. We will continue to maintain databases of available experiments; but users can also create their own (smaller) databases covering just the simulations that they are interested in. This example shows how to build your database.


In [2]:
%matplotlib inline
import cosima_cookbook as cc
from dask.distributed import Client

**First, create a database using this function:**

In [8]:
help(cc.database.create_database)

Help on function create_database in module cosima_cookbook.database:

create_database(db, debug=False)
    Create new database file with the target schema.
    
    We create a foreign key constraint on the ncfile column in
    the ncvars table, but it won't be enforced without `PRAGMA
    foreign_keys = 1' in sqlite.



We will use our default ACCESS-OM2-01 database by way of example:

In [3]:
db = '/g/data3/hh5/tmp/cosima/database/access-om2.db'

In [6]:
cc.database.create_database(db)

(<sqlalchemy.engine.base.Connection at 0x7f49c7f2ef98>,
 {'ncfiles': Table('ncfiles', MetaData(bind=None), Column('id', Integer(), table=<ncfiles>, primary_key=True, nullable=False), Column('index_time', DateTime(), table=<ncfiles>), Column('ncfile', Text(), table=<ncfiles>), Column('present', Boolean(), table=<ncfiles>), Column('experiment', Text(), table=<ncfiles>), Column('run', Integer(), table=<ncfiles>), Column('timeunits', Text(), table=<ncfiles>), Column('calendar', Text(), table=<ncfiles>), Column('time_start', Text(), table=<ncfiles>), Column('time_end', Text(), table=<ncfiles>), Column('frequency', Text(), table=<ncfiles>), schema=None),
  'ncvars': Table('ncvars', MetaData(bind=None), Column('id', Integer(), table=<ncvars>, primary_key=True, nullable=False), Column('ncfile', Integer(), ForeignKey('ncfiles.id'), table=<ncvars>, nullable=False), Column('variable', Text(), table=<ncvars>), Column('dimensions', Text(), table=<ncvars>), Column('chunking', Text(), table=<ncvars>)

**Second, start up a client** (required to build the index):

In [4]:
client = Client(n_workers=4)
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:34150 Dashboard: http://127.0.0.1:36146/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


**Now, you're ready to build the database** using the following function:

In [7]:
help(cc.database.build_index)

Help on function build_index in module cosima_cookbook.database:

build_index(directories, client, db, update=False, debug=False)
    Index all runs contained within a directory. Requires a distributed client for processing,
    and the filename of a database that's been created with the create_database() function.
    
    May scan for only new entries to add to database with the update flag.



In [9]:
dir_list=[#'/g/data3/hh5/tmp/cosima/access-om2-01',
          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_nogmredi6',
          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6',
          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_ryf8485_gmredi6',
#          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_ryf9091_gmredi6',
#          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_SAMx_0203_from_ryf9091_gmredi6',
#          '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_SAMx_9899_from_ryf9091_gmredi6',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_A',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_A_lastcycle',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1_lastcycle',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_core_nyf_spinup_A',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf8485_kds50_july_bkd_100',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf9091_kds50_july_bkd_100',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf0304_kds50_july_bkd_100',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf9091_spinup1_B1',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf8485_spinup1_B1',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1617_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1516_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1011_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf0203_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9899_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9192_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9091_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf0203_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1617_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9899_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9091_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9192_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1011_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf8485_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1516_from0304_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1617_from9091_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf0304_from9091_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9899_from9091_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf9192_from9091_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1011_from9091_kds50',
#          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf8485_from9091_kds50',
          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf0203_from9091_kds50',
          '/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_SAMextr_ryf1516_from9091_kds50']

cc.database.build_index(dir_list,client,db,update=True)




515/515

You can choose to supply a list of directories, or a single directory, to be included in your database. Note that this operation will take 10 minutes or so for the first build of the database, or if `update=False`. 